In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bh0yyfi2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bh0yyfi2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=1452ff82eb286f252f25320a66faabc2b6075bd36c5c3a2cb421cad6e5b6123b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8uufg0k4/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu

#include<stdio.h>
#define N 4

__global__ void add(int *a,int *b,int *c,int n)
{
    int id=threadIdx.x+ blockDim.x * blockIdx.x;
    
    if(id<n)
      c[id]=a[id]+b[id];
}

int main()
{
  int* a;
  int* b;
  int* c;
  
  a=(int *)malloc(sizeof(int)*N);
  b=(int *)malloc(sizeof(int)*N);
  c=(int *)malloc(sizeof(int)*N);
  
  for(int i=0;i<N;i++)
  {
      a[i]=i;
      b[i]=i+1;
  }
  
  for(int i=0;i<N;i++)
  {
      printf("%d\t",a[i]);
  }
   
  printf("\n");
  
  for(int i=0;i<N;i++)
  {
      printf("%d\t",b[i]);
  }
   
  printf("\n");
  
  
  int* d_a;
  int* d_b;
  int* d_c; 
  cudaMalloc(&d_a,sizeof(int)*N);
  cudaMalloc(&d_b,sizeof(int)*N);
  cudaMalloc(&d_c,sizeof(int)*N);
   
  cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,b,sizeof(int)*N,cudaMemcpyHostToDevice);
  
  add <<<2,N/2>>> (d_a,d_b,d_c,N);
  cudaDeviceSynchronize();
  
  cudaMemcpy(c,d_c,sizeof(int)*N,cudaMemcpyDeviceToHost);
  
  for(int i=0;i<N;i++)
  {
      printf("%d\t",c[i]);
  }
   
  //printf("\n");
  
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  free(a);
  free(b);
  free(c);
}


0	1	2	3	
1	2	3	4	
1	3	5	7	


In [0]:
%%cu

#include<stdio.h>
#define N 4

//matrix_vector

__global__ void vec_mat(int *a,int *b,int *c)
{
    int id=threadIdx.x;
    
    for(int i=0;i<N;i++)
      c[id]=c[id]+ (a[i]* *(b+ N*id+ i));
    
}

int main()
{
  int* a;
  int* b;
  int* c;
  
  a=(int *)malloc(sizeof(int)*N);
  b=(int *)malloc(sizeof(int)*N*N);
  c=(int *)malloc(sizeof(int)*N);
  
  for(int i=0;i<N;i++)
  {
      a[i]=i;
  }
    
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          *(b + i*N+ j)=i+j;
      }
  }
  
  for(int i=0;i<N;i++)
  {
      printf("%d\t",a[i]);
  }
   
  printf("\n");
  
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          printf("%d\t",*(b + i*N+ j));
      }
      printf("\n");
  }
   
  printf("\n");
  
  
  int* d_a;
  int* d_b;
  int* d_c; 
  cudaMalloc(&d_a,sizeof(int)*N);
  cudaMalloc(&d_b,sizeof(int)*N*N);
  cudaMalloc(&d_c,sizeof(int)*N);
   
  cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,b,sizeof(int)*N*N,cudaMemcpyHostToDevice);
  
  vec_mat <<<1,N>>> (d_a,d_b,d_c);
  cudaDeviceSynchronize();
  
  cudaMemcpy(c,d_c,sizeof(int)*N,cudaMemcpyDeviceToHost);
  
  for(int i=0;i<N;i++)
  {
      printf("%d\t",c[i]);
  }
   
  //printf("\n");
  
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  free(a);
  free(b);
  free(c);
}


0	1	2	3	
0	1	2	3	
1	2	3	4	
2	3	4	5	
3	4	5	6	

14	20	26	32	


In [0]:
%%cu

#include<stdio.h>
#define N 3

//matrix_mul

__global__ void matmul(int *a,int *b,int *c)
{
    int tx=threadIdx.x;
    int ty=threadIdx.y;
    int res=0;
    for(int i=0;i<N;i++)
    {
        int p= *(a+ ty*N +i);
        int q= *(b+ N*i + tx);
        res+=p*q;
    }
    *(c + N*ty+ tx)=res;
}

int main()
{
  int* a;
  int* b;
  int* c;
  
  a=(int *)malloc(sizeof(int)*N*N);
  b=(int *)malloc(sizeof(int)*N*N);
  c=(int *)malloc(sizeof(int)*N*N);
    
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          *(b + i*N+ j)=i+j;
          *(a + i*N+ j)=i;
      }
  }
  
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          printf("%d\t",*(a + i*N+ j));
      }
      printf("\n");
  }
    
  printf("\n");
  
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          printf("%d\t",*(b + i*N+ j));
      }
      printf("\n");
  }
   
  printf("\n");
  
  
  int* d_a;
  int* d_b;
  int* d_c; 
  cudaMalloc(&d_a,sizeof(int)*N*N);
  cudaMalloc(&d_b,sizeof(int)*N*N);
  cudaMalloc(&d_c,sizeof(int)*N*N);
   
  cudaMemcpy(d_a,a,sizeof(int)*N*N,cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,b,sizeof(int)*N*N,cudaMemcpyHostToDevice);
  
  dim3 grid(N,N);
  matmul <<<1,grid>>> (d_a,d_b,d_c);
  cudaDeviceSynchronize();
  
  cudaMemcpy(c,d_c,sizeof(int)*N*N,cudaMemcpyDeviceToHost);
  
  for(int i=0;i<N;i++)
  {
      for(int j=0;j<N;j++)
      {
          printf("%d\t",*(c + i*N+ j));
      }
      printf("\n");
  }
  
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  free(a);
  free(b);
  free(c);
}


0	0	0	
1	1	1	
2	2	2	

0	1	2	
1	2	3	
2	3	4	

0	0	0	
3	6	9	
6	12	18	

